https://shiny.ilykei.com/misha/ec/08_pacman/

In [1]:
import time
import random
import os
import json

import numpy as np

import tensorflow as tf
from tensorflow.keras.models import Sequential, clone_model
from tensorflow.keras.layers import Dense, InputLayer, Conv2D, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import CSVLogger, TensorBoard
import tensorflow.keras.backend as K
from tensorflow.keras.losses import Huber

from mini_pacman import PacmanGame

In [2]:
paramsfile = 'test_params.json'
with open(paramsfile, 'r') as file:
    read_params = json.load(file)
game_params = read_params['params']

env = PacmanGame(**game_params)

In [3]:
#env.render()
env.print_field()

╔════════╗
║       ♦║
║  ☺     ║
║ ▒▒▒ ▒♦ ║
║  ▒  ☢ ▒║
║▒     ☢ ║
║▒       ║
║    ▒  ♦║
║      ▒ ║
╚════════╝


In [4]:
game_params

{'field_shape': [8, 8],
 'nmonsters': 2,
 'ndiamonds': 3,
 'nwalls': 10,
 'monster_vision_range': 2,
 'max_moves': 100,
 'diamond_reward': 100,
 'survival_reward': 1}

In [5]:
# Render random-action game
obs = env.reset()
while not obs['end_game']:
    action = random.choice(obs['possible_actions'])
    obs = env.make_action(action)
    #env.render()
    env.print_field()
    time.sleep(0.5)

╔════════╗
║▒       ║
║▒  ▒    ║
║      ☢ ║
║        ║
║     ▒  ║
║ ☺ ♦  ▒☢║
║♦    ▒ ▒║
║▒▒  ♦▒  ║
╚════════╝
╔════════╗
║▒       ║
║▒  ▒    ║
║        ║
║      ☢ ║
║     ▒☢ ║
║ ☺ ♦  ▒ ║
║♦    ▒ ▒║
║▒▒  ♦▒  ║
╚════════╝
╔════════╗
║▒       ║
║▒  ▒    ║
║        ║
║      ☢☢║
║     ▒  ║
║ ☺ ♦  ▒ ║
║♦    ▒ ▒║
║▒▒  ♦▒  ║
╚════════╝
╔════════╗
║▒       ║
║▒  ▒    ║
║     ☢☢ ║
║        ║
║     ▒  ║
║   ♦  ▒ ║
║♦☺   ▒ ▒║
║▒▒  ♦▒  ║
╚════════╝
╔════════╗
║▒       ║
║▒  ▒    ║
║     ☢  ║
║       ☢║
║     ▒  ║
║   ♦  ▒ ║
║♦ ☺  ▒ ▒║
║▒▒  ♦▒  ║
╚════════╝
╔════════╗
║▒       ║
║▒  ▒    ║
║        ║
║     ☢  ║
║     ▒☢ ║
║   ♦  ▒ ║
║♦ ☺  ▒ ▒║
║▒▒  ♦▒  ║
╚════════╝
╔════════╗
║▒       ║
║▒  ▒    ║
║        ║
║    ☢ ☢ ║
║     ▒  ║
║   ♦  ▒ ║
║♦ ☺  ▒ ▒║
║▒▒  ♦▒  ║
╚════════╝
╔════════╗
║▒       ║
║▒  ▒    ║
║    ☢   ║
║       ☢║
║     ▒  ║
║  ☺♦  ▒ ║
║♦    ▒ ▒║
║▒▒  ♦▒  ║
╚════════╝
╔════════╗
║▒       ║
║▒  ▒☢   ║
║        ║
║        ║
║  ☺  ▒☢ ║
║   ♦  ▒ ║
║♦    ▒ ▒║
║▒▒  ♦▒  ║
╚════════╝
╔════════╗

In [6]:
env.close()

In [7]:
game_scores=[]
for one_game in range(100):
    obs = env.reset()
    while not obs['end_game']:    
        action = random.choice(obs['possible_actions'])
        obs = env.make_action(action)
    game_scores.append(obs['total_score'])
print('Mean: ',np.mean(game_scores),'\nMedian: ',np.median(game_scores))

Mean:  34.94 
Median:  3.5


In [8]:
obs

{'reward': 0,
 'total_score': 1,
 'end_game': True,
 'player': (0, 5),
 'monsters': [(0, 5), (2, 0)],
 'diamonds': [(2, 1), (3, 0), (6, 7)],
 'walls': [(0, 1),
  (0, 2),
  (3, 2),
  (4, 3),
  (5, 0),
  (6, 0),
  (6, 4),
  (6, 5),
  (7, 0),
  (7, 5)],
 'possible_actions': [1, 2, 3, 4, 5, 6]}

In [10]:
# MAY APPEND ANY HAND-MADE FEATURES HERE

def get_state(obs):
    v = np.zeros((14,14,3))
    v[:,:,2] = 1
    v[3:11,3:11,2] = 0
    x_play, y_play = obs['player']
    x_play += 3
    y_play += 3
    for x, y in obs['monsters']:
        v[x+3,y+3,0] = 1
    for x, y in obs['diamonds']:
        v[x+3,y+3,1] = 1
    for x, y in obs['walls']:
        v[x+3,y+3,2] = 1
    return v[(x_play-3):(x_play+4),(y_play-3):(y_play+4),:]
#     return v

In [12]:
def create_dqn_model(input_shape, nb_actions):
    model = Sequential()
    model.add(InputLayer(input_shape=input_shape))
    model.add(Flatten())
    for i in range(2):
        model.add(Dense(units=64, activation='relu'))
    model.add(Dense(nb_actions, activation='linear'))
    return model

In [13]:
input_shape = state.shape
nb_actions = 9
print('input_shape: ',input_shape)
print('nb_actions: ',nb_actions)

online_network = create_dqn_model(input_shape, nb_actions)
target_network = clone_model(online_network)
target_network.set_weights(online_network.get_weights())

input_shape:  (7, 7, 3)
nb_actions:  9


In [14]:
name = 'minipacman_dqn_cnn'  # used in naming files (weights, logs, etc)
n_steps = 500_000        # total number of training steps (= n_epochs)
warmup = 10_000          # start training after warmup iterations
training_interval = 1  # period (in actions) between training steps
save_steps = n_steps//10  # period (in training steps) between storing weights to file
copy_steps = 5000       # period (in training steps) between updating target_network weights
gamma = 0.99           # discount rate
batch_size = 64        # size of minibatch that is taken randomly from replay memory every training step
double_dqn = False     # whether to use Double-DQN approach or simple DQN (see above)
# eps-greedy parameters: we slowly decrease epsilon from eps_max to eps_min in eps_decay_steps
eps_max = 1.0
eps_min = 0.05
eps_decay_steps = n_steps//2

# optimizer params
learning_rate = 0.00025
clipnorm = 1.0

In [15]:
from collections import deque

replay_memory_maxlen = 1_000_000
replay_memory = deque([], maxlen=replay_memory_maxlen)

In [16]:
def epsilon_greedy(q_values, epsilon, n_outputs):
    if random.random() < epsilon:
        return random.randrange(n_outputs)  # random action
    else:
        return np.argmax(q_values)          # q-optimal action

In [17]:
def mean_q(y_true, y_pred):
    return K.mean(K.max(y_pred, axis=-1))

optimizer = Adam(learning_rate, clipnorm=clipnorm)
online_network.compile(optimizer, loss=Huber(), metrics=[mean_q])

In [18]:
if not os.path.exists(name):
    os.makedirs(name)

In [26]:
# counters:
# step = 0          # training step counter (= epoch counter)
# iteration = 0     # frames counter
# episodes = 0      # game episodes counter
# done = True       # indicator that env needs to be reset

episode_scores = []  # collect total scores in this list and log it later

while step < n_steps:
    if done:  # game over, restart it
        obs = env.reset()
        state = get_state(env.get_obs())
        score = 0  # reset score for current episode
        episodes += 1

    # Online network evaluates what to do
    iteration += 1
    # calculate q-values using online network:
    q_values = online_network.predict(np.array([state]))[0]
    # select epsilon (which linearly decreases over training steps):
    epsilon = max(eps_min, eps_max - (eps_max-eps_min) * step/eps_decay_steps)
    action = epsilon_greedy(q_values, epsilon, nb_actions)
    # Play:
    obs = env.make_action(action+1)
    reward = obs['reward']
    score += reward
    done = obs['end_game']
    if done:
        episode_scores.append(score)
    next_state = get_state(env.get_obs())
    # Let's memorize what has just happened:
    replay_memory.append((state, action, reward, next_state, done))
    state = next_state

    if iteration >= warmup and iteration % training_interval == 0:
        # learning branch
        step += 1
        minibatch = random.sample(replay_memory, batch_size)
        replay_state = np.array([x[0] for x in minibatch])
        replay_action = np.array([x[1] for x in minibatch])
        replay_rewards = np.array([x[2] for x in minibatch])
        replay_next_state = np.array([x[3] for x in minibatch])
        replay_done = np.array([x[4] for x in minibatch], dtype=int)

        # calculate targets (see above for details)
        if double_dqn == False:
            # DQN approach
            target_for_action = replay_rewards + (1-replay_done) * gamma * \
                                    np.amax(target_network.predict(replay_next_state), axis=1)
        else:
            # Double DQN approach
            best_actions = np.argmax(online_network.predict(replay_next_state), axis=1)
            target_for_action = replay_rewards + (1-replay_done) * gamma * \
                                    target_network.predict(replay_next_state)[np.arange(batch_size), best_actions]

        # targets coincide with predictions ...
        target = online_network.predict(replay_state)
        #...except for targets with actions from the replay batch:
        target[np.arange(batch_size), replay_action] = target_for_action
        
        # Train online network:
        online_network.fit(replay_state, target, epochs=step, 
                           verbose=1, initial_epoch=step-1)

        # Periodically copy online network weights to target network
        if step % copy_steps == 0:
            target_network.set_weights(online_network.get_weights())
        # And save model
        if step % save_steps == 0:
            online_network.save_weights(os.path.join(name, f'{name}_{step}'))

Epoch 31023/31023
2/2 [==============================] - 0s 1ms/step - loss: 2.7437 - mean_q: 143.7368
Epoch 31024/31024
2/2 [==============================] - 0s 2ms/step - loss: 2.2846 - mean_q: 129.6381
Epoch 31025/31025
2/2 [==============================] - 0s 3ms/step - loss: 2.2053 - mean_q: 118.4386
Epoch 31026/31026
2/2 [==============================] - 0s 2ms/step - loss: 1.9617 - mean_q: 123.9286
Epoch 31027/31027
2/2 [==============================] - 0s 2ms/step - loss: 2.2972 - mean_q: 132.7537
Epoch 31028/31028
2/2 [==============================] - 0s 2ms/step - loss: 2.7716 - mean_q: 150.0834
Epoch 31029/31029
2/2 [==============================] - 0s 3ms/step - loss: 2.3635 - mean_q: 143.7655
Epoch 31030/31030
2/2 [==============================] - 0s 2ms/step - loss: 2.4001 - mean_q: 145.0720
Epoch 31031/31031
2/2 [==============================] - 0s 1ms/step - loss: 2.3315 - mean_q: 148.1042
Epoch 31032/31032
2/2 [==============================] - 0s 3ms/step - lo

2/2 [==============================] - 0s 3ms/step - loss: 2.8069 - mean_q: 131.1573
Epoch 31103/31103
2/2 [==============================] - 0s 3ms/step - loss: 2.9078 - mean_q: 142.7309
Epoch 31104/31104
2/2 [==============================] - 0s 3ms/step - loss: 3.2889 - mean_q: 147.1166
Epoch 31105/31105
2/2 [==============================] - 0s 4ms/step - loss: 2.2093 - mean_q: 136.9965
Epoch 31106/31106
2/2 [==============================] - 0s 4ms/step - loss: 2.4479 - mean_q: 141.0609
Epoch 31107/31107
2/2 [==============================] - 0s 4ms/step - loss: 2.4681 - mean_q: 154.5554
Epoch 31108/31108
2/2 [==============================] - 0s 3ms/step - loss: 3.2125 - mean_q: 164.6100
Epoch 31109/31109
2/2 [==============================] - 0s 3ms/step - loss: 3.0004 - mean_q: 141.1405
Epoch 31110/31110
2/2 [==============================] - 0s 4ms/step - loss: 2.8935 - mean_q: 134.3820
Epoch 31111/31111
2/2 [==============================] - 0s 3ms/step - loss: 2.5770 - mean_

2/2 [==============================] - 0s 1ms/step - loss: 2.9034 - mean_q: 143.3171
Epoch 31182/31182
2/2 [==============================] - 0s 1ms/step - loss: 2.4709 - mean_q: 139.5894
Epoch 31183/31183
2/2 [==============================] - 0s 2ms/step - loss: 2.9064 - mean_q: 144.7662
Epoch 31184/31184
2/2 [==============================] - 0s 2ms/step - loss: 3.3161 - mean_q: 124.9240
Epoch 31185/31185
2/2 [==============================] - 0s 2ms/step - loss: 2.8173 - mean_q: 152.7240
Epoch 31186/31186
2/2 [==============================] - 0s 3ms/step - loss: 2.4214 - mean_q: 135.4413
Epoch 31187/31187
2/2 [==============================] - 0s 3ms/step - loss: 2.5746 - mean_q: 131.8422
Epoch 31188/31188
2/2 [==============================] - 0s 2ms/step - loss: 2.0550 - mean_q: 126.7225
Epoch 31189/31189
2/2 [==============================] - 0s 2ms/step - loss: 2.9957 - mean_q: 147.8395
Epoch 31190/31190
2/2 [==============================] - 0s 0s/step - loss: 2.3403 - mean_q

2/2 [==============================] - 0s 2ms/step - loss: 3.1216 - mean_q: 132.5723
Epoch 31261/31261
2/2 [==============================] - 0s 1ms/step - loss: 2.7938 - mean_q: 142.1494
Epoch 31262/31262
2/2 [==============================] - 0s 2ms/step - loss: 3.7020 - mean_q: 143.2803
Epoch 31263/31263
2/2 [==============================] - 0s 2ms/step - loss: 3.0305 - mean_q: 144.8648
Epoch 31264/31264
2/2 [==============================] - 0s 2ms/step - loss: 2.9211 - mean_q: 147.1389
Epoch 31265/31265
2/2 [==============================] - 0s 2ms/step - loss: 2.8528 - mean_q: 152.1575
Epoch 31266/31266
2/2 [==============================] - 0s 3ms/step - loss: 2.4758 - mean_q: 122.3675
Epoch 31267/31267
2/2 [==============================] - 0s 3ms/step - loss: 3.1034 - mean_q: 132.6725
Epoch 31268/31268
2/2 [==============================] - 0s 2ms/step - loss: 2.0598 - mean_q: 140.0251
Epoch 31269/31269
2/2 [==============================] - 0s 95us/step - loss: 2.7493 - mean

2/2 [==============================] - 0s 1ms/step - loss: 2.7765 - mean_q: 131.6517
Epoch 31340/31340
2/2 [==============================] - 0s 3ms/step - loss: 2.3605 - mean_q: 142.0201
Epoch 31341/31341
2/2 [==============================] - 0s 2ms/step - loss: 2.7942 - mean_q: 155.0582
Epoch 31342/31342
2/2 [==============================] - 0s 1ms/step - loss: 2.6777 - mean_q: 122.1405
Epoch 31343/31343
2/2 [==============================] - 0s 1ms/step - loss: 2.1972 - mean_q: 140.3504
Epoch 31344/31344
2/2 [==============================] - 0s 3ms/step - loss: 2.5891 - mean_q: 144.8609
Epoch 31345/31345
2/2 [==============================] - 0s 2ms/step - loss: 2.6887 - mean_q: 113.8688
Epoch 31346/31346
2/2 [==============================] - 0s 1ms/step - loss: 2.4765 - mean_q: 145.1566
Epoch 31347/31347
2/2 [==============================] - 0s 0s/step - loss: 2.5041 - mean_q: 148.4818
Epoch 31348/31348
2/2 [==============================] - 0s 2ms/step - loss: 2.8385 - mean_q

...

2/2 [==============================] - 0s 1ms/step - loss: 3.7661 - mean_q: 170.3696
Epoch 45244/45244
2/2 [==============================] - 0s 3ms/step - loss: 2.9670 - mean_q: 176.8108
Epoch 45245/45245
2/2 [==============================] - 0s 3ms/step - loss: 4.4436 - mean_q: 177.4317
Epoch 45246/45246
2/2 [==============================] - 0s 2ms/step - loss: 3.9317 - mean_q: 152.9346
Epoch 45247/45247
2/2 [==============================] - 0s 3ms/step - loss: 3.7702 - mean_q: 178.1980
Epoch 45248/45248
2/2 [==============================] - 0s 1ms/step - loss: 3.6710 - mean_q: 186.5972
Epoch 45249/45249
2/2 [==============================] - 0s 3ms/step - loss: 3.4262 - mean_q: 162.8949
Epoch 45250/45250
2/2 [==============================] - 0s 3ms/step - loss: 3.1701 - mean_q: 167.2294
Epoch 45251/45251
2/2 [==============================] - 0s 3ms/step - loss: 2.5473 - mean_q: 169.5963
Epoch 45252/45252
2/2 [==============================] - 0s 3ms/step - loss: 3.9379 - mean_

2/2 [==============================] - 0s 3ms/step - loss: 3.5273 - mean_q: 153.5551
Epoch 45323/45323
2/2 [==============================] - 0s 2ms/step - loss: 2.7003 - mean_q: 188.1297
Epoch 45324/45324
2/2 [==============================] - 0s 3ms/step - loss: 3.1939 - mean_q: 181.0904
Epoch 45325/45325
2/2 [==============================] - 0s 2ms/step - loss: 3.5762 - mean_q: 176.2415
Epoch 45326/45326
2/2 [==============================] - 0s 3ms/step - loss: 2.9500 - mean_q: 169.1727
Epoch 45327/45327
2/2 [==============================] - 0s 3ms/step - loss: 3.7594 - mean_q: 173.5633
Epoch 45328/45328
2/2 [==============================] - 0s 3ms/step - loss: 2.5646 - mean_q: 172.0014
Epoch 45329/45329
2/2 [==============================] - 0s 3ms/step - loss: 3.5597 - mean_q: 183.4107
Epoch 45330/45330
2/2 [==============================] - 0s 3ms/step - loss: 3.0897 - mean_q: 149.4940
Epoch 45331/45331
2/2 [==============================] - 0s 3ms/step - loss: 4.0696 - mean_

2/2 [==============================] - 0s 1ms/step - loss: 3.4793 - mean_q: 157.6076
Epoch 45402/45402
2/2 [==============================] - 0s 1ms/step - loss: 2.9929 - mean_q: 168.2636
Epoch 45403/45403
2/2 [==============================] - 0s 0s/step - loss: 3.5173 - mean_q: 161.6843
Epoch 45404/45404
2/2 [==============================] - 0s 0s/step - loss: 2.9618 - mean_q: 165.0728
Epoch 45405/45405
2/2 [==============================] - 0s 5ms/step - loss: 3.3318 - mean_q: 180.0977
Epoch 45406/45406
2/2 [==============================] - 0s 2ms/step - loss: 2.5066 - mean_q: 174.0111
Epoch 45407/45407
2/2 [==============================] - 0s 3ms/step - loss: 4.4586 - mean_q: 154.8451
Epoch 45408/45408
2/2 [==============================] - 0s 3ms/step - loss: 3.6701 - mean_q: 169.8190
Epoch 45409/45409
2/2 [==============================] - 0s 8ms/step - loss: 3.8991 - mean_q: 178.8897
Epoch 45410/45410
2/2 [==============================] - 0s 4ms/step - loss: 2.7710 - mean_q:

2/2 [==============================] - 0s 3ms/step - loss: 3.8162 - mean_q: 155.2508
Epoch 45481/45481
2/2 [==============================] - 0s 3ms/step - loss: 3.6496 - mean_q: 175.8566
Epoch 45482/45482
2/2 [==============================] - 0s 3ms/step - loss: 3.4402 - mean_q: 163.4956
Epoch 45483/45483
2/2 [==============================] - 0s 1ms/step - loss: 2.9645 - mean_q: 171.4554
Epoch 45484/45484
2/2 [==============================] - 0s 3ms/step - loss: 2.9376 - mean_q: 174.4617
Epoch 45485/45485
2/2 [==============================] - 0s 3ms/step - loss: 3.8941 - mean_q: 177.6262
Epoch 45486/45486
2/2 [==============================] - 0s 0s/step - loss: 3.6880 - mean_q: 184.3327
Epoch 45487/45487
2/2 [==============================] - 0s 3ms/step - loss: 2.8256 - mean_q: 174.0363
Epoch 45488/45488
2/2 [==============================] - 0s 3ms/step - loss: 3.4121 - mean_q: 194.5470
Epoch 45489/45489
2/2 [==============================] - 0s 3ms/step - loss: 3.8160 - mean_q

KeyboardInterrupt: 

In [28]:
online_network.save_weights(os.path.join(name, f'{name}_final'))
#online_network.load(os.path.join(name, f'{name}_final.mdl'))
#online_network.load_weights('weights_last.h5f')

In [20]:
def dqn_strategy(obs):
    state = get_state(obs)
    q_values = online_network.predict(np.array([state]))[0]
    action = epsilon_greedy(q_values, 0, nb_actions)
    return action + 1

In [27]:
from mini_pacman import test, random_strategy, naive_strategy

test(strategy=dqn_strategy, log_file='test_pacman_log.json', verbose=True)

Game 0/500, score=17
Game 1/500, score=59
Game 2/500, score=204
Game 3/500, score=219
Game 4/500, score=427
Game 5/500, score=208
Game 6/500, score=21
Game 7/500, score=7
Game 8/500, score=11
Game 9/500, score=7
Game 10/500, score=700
Game 11/500, score=230
Game 12/500, score=742
Game 13/500, score=149
Game 14/500, score=210
Game 15/500, score=1
Game 16/500, score=1000
Game 17/500, score=33
Game 18/500, score=223
Game 19/500, score=3
Game 20/500, score=541
Game 21/500, score=119
Game 22/500, score=1100
Game 23/500, score=476
Game 24/500, score=15
Game 25/500, score=350
Game 26/500, score=116
Game 27/500, score=119
Game 28/500, score=898
Game 29/500, score=400
Game 30/500, score=400
Game 31/500, score=120
Game 32/500, score=115
Game 33/500, score=97
Game 34/500, score=138
Game 35/500, score=382
Game 36/500, score=0
Game 37/500, score=333
Game 38/500, score=17
Game 39/500, score=119
Game 40/500, score=116
Game 41/500, score=9
Game 42/500, score=492
Game 43/500, score=139
Game 44/500, sco

Game 353/500, score=281
Game 354/500, score=257
Game 355/500, score=500
Game 356/500, score=254
Game 357/500, score=343
Game 358/500, score=448
Game 359/500, score=7
Game 360/500, score=226
Game 361/500, score=36
Game 362/500, score=61
Game 363/500, score=467
Game 364/500, score=528
Game 365/500, score=212
Game 366/500, score=2
Game 367/500, score=211
Game 368/500, score=100
Game 369/500, score=104
Game 370/500, score=128
Game 371/500, score=117
Game 372/500, score=149
Game 373/500, score=123
Game 374/500, score=110
Game 375/500, score=8
Game 376/500, score=10
Game 377/500, score=127
Game 378/500, score=396
Game 379/500, score=1
Game 380/500, score=462
Game 381/500, score=7
Game 382/500, score=15
Game 383/500, score=206
Game 384/500, score=131
Game 385/500, score=4
Game 386/500, score=0
Game 387/500, score=106
Game 388/500, score=500
Game 389/500, score=6
Game 390/500, score=34
Game 391/500, score=324
Game 392/500, score=0
Game 393/500, score=7
Game 394/500, score=221
Game 395/500, sco

166.0